In [104]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
pd.options.display.max_columns = None

## Health Data by County

In [ ]:
# CDC LOCALS health data
# https://data.cdc.gov/500-Cities-Places/PLACES-ZCTA-Data-GIS-Friendly-Format-2023-release/kee5-23sr/about_data
# https://dev.socrata.com/foundry/data.cdc.gov/swc5-untb info about data

In [128]:
df_health1 = pd.read_csv("../data/raw/PLACES__Local_Data_for_Better_Health__County_Data_2022_release_20240102.csv")

In [129]:
df_health1.Year.unique()

array([2020, 2019])

In [130]:
df_health2 = pd.read_csv("../data/raw/PLACES__Local_Data_for_Better_Health__County_Data_2023_release_20240102.csv")

In [131]:
df_health2.Year.unique()

array([2021, 2020])

In [132]:
df_health1.columns

Index(['Year', 'StateAbbr', 'StateDesc', 'LocationName', 'DataSource',
       'Category', 'Measure', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit', 'TotalPopulation',
       'LocationID', 'CategoryID', 'MeasureId', 'DataValueTypeID',
       'Short_Question_Text', 'Geolocation'],
      dtype='object')

In [133]:
unique_lists_cols = ['Year', 'DataSource',
       'Category', 'Measure', 'Data_Value_Unit', 'Data_Value_Type',
        'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'LocationID', 'CategoryID', 'MeasureId', 'DataValueTypeID',
       'Short_Question_Text']

In [134]:
for col in unique_lists_cols:
    print(col)
    print(df_health1[col].unique()[0:4])

Year
[2020 2019]
DataSource
['BRFSS']
Category
['Prevention' 'Health Outcomes' 'Health Risk Behaviors' 'Health Status']
Measure
['Current lack of health insurance among adults aged 18-64 years'
 'Diagnosed diabetes among adults aged >=18 years'
 'Visits to doctor for routine checkup within the past year among adults aged >=18 years'
 'Cervical cancer screening among adult women aged 21-65 years']
Data_Value_Unit
['%']
Data_Value_Type
['Crude prevalence' 'Age-adjusted prevalence']
Data_Value_Footnote_Symbol
[nan]
Data_Value_Footnote
[nan]
LocationID
[  59 1121 1123 1131]
CategoryID
['PREVENT' 'HLTHOUT' 'RISKBEH' 'HLTHSTAT']
MeasureId
['ACCESS2' 'DIABETES' 'CHECKUP' 'CERVICAL']
DataValueTypeID
['CrdPrv' 'AgeAdjPrv']
Short_Question_Text
['Health Insurance' 'Diabetes' 'Annual Checkup'
 'Cervical Cancer Screening']


In [135]:
for col in df_health1.columns:
    print(col)
    print(df_health1[col].nunique())

Year
2
StateAbbr
52
StateDesc
52
LocationName
1840
DataSource
1
Category
4
Measure
30
Data_Value_Unit
1
Data_Value_Type
2
Data_Value
903
Data_Value_Footnote_Symbol
0
Data_Value_Footnote
0
Low_Confidence_Limit
900
High_Confidence_Limit
911
TotalPopulation
3068
LocationID
3144
CategoryID
4
MeasureId
30
DataValueTypeID
2
Short_Question_Text
30
Geolocation
3143


In [136]:
df_health1.StateAbbr.unique()

array(['US', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'FL', 'GA', 'ID',
       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS',
       'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
       'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA',
       'WY', 'WA', 'WV', 'WI', 'DE', 'DC', 'HI', 'ME'], dtype=object)

In [137]:
df_health1.LocationName.nunique()

1840

In [138]:
df_health1.DataSource.unique()

array(['BRFSS'], dtype=object)

In [139]:
df_health1.Category.unique()

array(['Prevention', 'Health Outcomes', 'Health Risk Behaviors',
       'Health Status'], dtype=object)

## Economic Data by County

In [95]:
#Economic Data - by county
# https://www.bea.gov/data/gdp/gdp-county-metro-and-other-areas
file_path = "../data/raw/lagdp1223.xlsx"
df = pd.read_excel(file_path, skiprows=3,skipfooter=4,names=['Location','GDP_2019','GDP_2020','GDP_2021','GDP_2022','GDP_RankInState_2022','PctChange_2020','PctChange_2021','PctChange_2022','PctChange_RankInState_2022'])

In [96]:
# Initialize an empty list to hold the state names as we go
states = []

# Initialize the current state variable
current_state = None

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Check if the Location is USA
    if row['Location'] == 'United States':
            # Set both State and County as 'Total'
            df.at[index, 'State'] = 'TOTAL'
            df.at[index, 'County'] = 'TOTAL'
            current_state = 'United States'  # Set the current state to 'United States'
            states.append(current_state)  # Add to our list of states


    elif pd.isna(row['Location']):
        # Set the next row's Location as the current state
        # Check if the next row exists to avoid IndexError at the end of the DataFrame
        if index + 1 < len(df):
            current_state = df.at[index + 1, 'Location']
            states.append(current_state)  # Add the state to our list
    else:
        # If it's not NaN and not a state (since states were added to the list),
        # it must be a county, so we assign the current state to it
        if row['Location'] not in states:
            df.at[index, 'State'] = current_state
            df.at[index, 'County'] = row['Location']
        else:
            # It's a state, so we assign it as 'Total' to the County column
            df.at[index, 'State'] = row['Location']
            df.at[index, 'County'] = 'TOTAL'

# Drop the NaN rows from the DataFrame as they are just separators
df.dropna(subset=['Location'], inplace=True)
df.drop(columns=['Location'],inplace=True)
# The DataFrame now has 'State' and 'County' for each row
df = df[['State', 'County'] + [c for c in df.columns if c not in ['State', 'County']]]



In [97]:
df.head()  # Display the first few rows to check our work

,State,County,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_RankInState_2022,PctChange_2020,PctChange_2021,PctChange_2022,PctChange_RankInState_2022
0,TOTAL,TOTAL,20573580103,20106509888,21271676784,21668520913,--,-2.3,5.8,1.9,--
2,Alabama,TOTAL,224944577,222081439,231892626,235807320,--,-1.3,4.4,1.7,--
3,Alabama,Autauga,1730861,1722438,1727818,1929264,23,-0.5,0.3,11.7,1
4,Alabama,Baldwin,8148786,8102009,8738819,8924207,7,-0.6,7.9,2.1,27
5,Alabama,Barbour,762557,731636,747888,745349,42,-4.1,2.2,-0.3,43


In [98]:
df.tail()

,State,County,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_RankInState_2022,PctChange_2020,PctChange_2021,PctChange_2022,PctChange_RankInState_2022
3214,Wyoming,Sweetwater,3695022,3405807,3288513,3095421,5,-7.8,-3.4,-5.9,19
3215,Wyoming,Teton,2620496,2659859,3591537,3507901,4,1.5,35,-2.3,16
3216,Wyoming,Uinta,882964,783682,800735,813461,13,-11.2,2.2,1.6,7
3217,Wyoming,Washakie,346359,344531,346237,342039,19,-0.5,0.5,-1.2,12
3218,Wyoming,Weston,303330,286131,285672,279273,21,-5.7,-0.2,-2.2,13


In [106]:
df[df.County=='Ware']

,State,County,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_RankInState_2022,PctChange_2020,PctChange_2021,PctChange_2022,PctChange_RankInState_2022
558,Georgia,Ware,1415915,1330060,1333672,1338301,56,-6.1,0.3,0.3,100


In [110]:
df.County.nunique()

array(['GA', 'IN', 'IA', 'TX', 'AL', 'AK', 'CA', 'AR', 'US', 'AZ', 'IL',
       'ID', 'KS', 'KY', 'CO', 'CT', 'DE', 'FL', 'DC', 'HI', 'LA', 'MI',
       'MN', 'MS', 'MD', 'MA', 'ME', 'MT', 'NE', 'MO', 'NY', 'ND', 'OR',
       'OH', 'OK', 'PA', 'NV', 'NC', 'NJ', 'NM', 'NH', 'SD', 'SC', 'TN',
       'RI', 'VA', 'UT', 'WI', 'WY', 'WV', 'VT', 'WA'], dtype=object)

## Food Access Data by County

In [ ]:
# food access data by county
# https://www.ers.usda.gov/data-products/food-environment-atlas/data-access-and-documentation-downloads/#Current%20Version
# for example has rate of fast food per county

In [140]:
df_food = pd.read_csv("../data/raw/FoodEnvironmentAtlas/StateAndCountyData.csv")

In [141]:
df_food.head()

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga,LACCESS_POP10,18428.439690
1,1001,AL,Autauga,LACCESS_POP15,17496.693040
2,1001,AL,Autauga,PCH_LACCESS_POP_10_15,-5.056026
3,1001,AL,Autauga,PCT_LACCESS_POP10,33.769657
4,1001,AL,Autauga,PCT_LACCESS_POP15,32.062255


In [142]:
df_food.State.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', ' AK', ' AZ', ' AR',
       ' CA', ' CO', ' CT', ' DE', ' DC', ' FL', ' GA', ' HI', ' ID',
       ' IL', ' IN', ' IA', ' KS', ' KY', ' LA', ' ME', ' MD', ' MA',
       ' MI', ' MN', ' MS', ' MO', ' MT', ' NE', ' NV', ' NH', ' NJ',
       ' NM', ' NY', ' NC', ' ND', ' OH', ' OK', ' OR', ' PA', ' RI',
       ' SC', ' SD', ' TN', ' TX', ' UT', ' VT', ' VA', ' WA', ' WV',
       ' WI', ' WY'], dtype=object)

In [143]:
df_food.County.nunique()

3708

In [144]:
df_food.Variable_Code.unique()

array(['LACCESS_POP10', 'LACCESS_POP15', 'PCH_LACCESS_POP_10_15',
       'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15', 'LACCESS_LOWI10',
       'LACCESS_LOWI15', 'PCH_LACCESS_LOWI_10_15', 'PCT_LACCESS_LOWI10',
       'PCT_LACCESS_LOWI15', 'LACCESS_HHNV10', 'LACCESS_HHNV15',
       'PCH_LACCESS_HHNV_10_15', 'PCT_LACCESS_HHNV10',
       'PCT_LACCESS_HHNV15', 'LACCESS_SNAP15', 'PCT_LACCESS_SNAP15',
       'LACCESS_CHILD10', 'LACCESS_CHILD15', 'LACCESS_CHILD_10_15',
       'PCT_LACCESS_CHILD10', 'PCT_LACCESS_CHILD15', 'LACCESS_SENIORS10',
       'LACCESS_SENIORS15', 'PCH_LACCESS_SENIORS_10_15',
       'PCT_LACCESS_SENIORS10', 'PCT_LACCESS_SENIORS15',
       'LACCESS_WHITE15', 'PCT_LACCESS_WHITE15', 'LACCESS_BLACK15',
       'PCT_LACCESS_BLACK15', 'LACCESS_HISP15', 'PCT_LACCESS_HISP15',
       'LACCESS_NHASIAN15', 'PCT_LACCESS_NHASIAN15', 'LACCESS_NHNA15',
       'PCT_LACCESS_NHNA15', 'LACCESS_NHPI15', 'PCT_LACCESS_NHPI15',
       'LACCESS_MULTIR15', 'PCT_LACCESS_MULTIR15', 'GROC11', 'GROC16',


In [145]:
df_food.Variable_Code.nunique()

332

In [146]:
df_food[df_food.County=='Ware']

,FIPS,State,County,Variable_Code,Value
21882,13299,GA,Ware,LACCESS_POP10,13610.323490
21883,13299,GA,Ware,LACCESS_POP15,14253.192790
21884,13299,GA,Ware,PCH_LACCESS_POP_10_15,4.723395
21885,13299,GA,Ware,PCT_LACCESS_POP10,37.481613
21886,13299,GA,Ware,PCT_LACCESS_POP15,39.252018
...,...,...,...,...,...
783275,13299,GA,Ware,PERPOV10,1.000000
783276,13299,GA,Ware,CHILDPOVRATE15,45.600000
783277,13299,GA,Ware,PERCHLDPOV10,1.000000
783278,13299,GA,Ware,METRO13,0.000000


In [152]:
df_food[(df_food.County=='Richmond') & (df_food.State=='VA') & (df_food.Variable_Code.str.startswith('FFR'))]

,FIPS,State,County,Variable_Code,Value
285716,51159,VA,Richmond,FFR11,4.000000
285717,51159,VA,Richmond,FFR16,2.000000
285719,51159,VA,Richmond,FFRPTH11,0.434452
285720,51159,VA,Richmond,FFRPTH16,0.227687
286516,51760,VA,Richmond,FFR11,179.000000
286517,51760,VA,Richmond,FFR16,188.000000
286519,51760,VA,Richmond,FFRPTH11,0.867206
286520,51760,VA,Richmond,FFRPTH16,0.834487


In [154]:
df_food[df_food.Variable_Code=='FFRPTH16'].sort_values('Value',ascending=False)

,FIPS,State,County,Variable_Code,Value
244332,8111,CO,San Juan,FFRPTH16,5.805515
286440,51720,VA,Norton,FFRPTH16,3.510532
284094,49033,UT,Rich,FFRPTH16,3.469211
286200,51600,VA,Fairfax,FFRPTH16,3.399505
286264,51640,VA,Galax,FFRPTH16,2.250900
...,...,...,...,...,...
253968,20033,KS,Comanche,FFRPTH16,0.000000
265652,30079,MT,Prairie,FFRPTH16,0.000000
273948,40053,OK,Grant,FFRPTH16,0.000000
277654,46057,SD,Hamlin,FFRPTH16,0.000000


,FIPS,State,County,Variable_Code,Value
15864,13001,GA,Appling,LACCESS_POP10,3655.485669
15865,13001,GA,Appling,LACCESS_POP15,3671.279857
15866,13001,GA,Appling,PCH_LACCESS_POP_10_15,0.432068
15867,13001,GA,Appling,PCT_LACCESS_POP10,20.045436
15868,13001,GA,Appling,PCT_LACCESS_POP15,20.132046
...,...,...,...,...,...
781070,13001,GA,Appling,PERPOV10,0.000000
781071,13001,GA,Appling,CHILDPOVRATE15,34.200000
781072,13001,GA,Appling,PERCHLDPOV10,1.000000
781073,13001,GA,Appling,METRO13,0.000000


In [155]:
df_food[df_food.Variable_Code=='FFRPTH16'].sort_values('Value',ascending=False)[0:20]

,FIPS,State,County,Variable_Code,Value
244332,8111,CO,San Juan,FFRPTH16,5.805515
286440,51720,VA,Norton,FFRPTH16,3.510532
284094,49033,UT,Rich,FFRPTH16,3.469211
286200,51600,VA,Fairfax,FFRPTH16,3.399505
286264,51640,VA,Galax,FFRPTH16,2.250900
286216,51610,VA,Falls Church,FFRPTH16,2.235362
283232,48433,TX,Stonewall,FFRPTH16,2.139800
277794,46075,SD,Jones,FFRPTH16,2.139037
286136,51570,VA,Colonial Heights,FFRPTH16,2.097268
286648,51840,VA,Winchester,FFRPTH16,2.031488


In [157]:
df_food[df_food.Variable_Code=='FFRPTH16'].County.nunique()

1832

In [158]:
df_food.County.nunique()

3708

In [159]:
df_food_variables = pd.read_csv("../data/raw/FoodEnvironmentAtlas/VariableList.csv")

In [160]:
df_food_variables.head()

,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Variable_Code,Geography,Units
0,"Population, low access to store, 2010",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP10,CNTY10,Count
1,"Population, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Overall,LACCESS_POP15,CNTY10,Count
2,"Population, low access to store (% change), 20...",Access and Proximity to Grocery Store,ACCESS,Overall,PCH_LACCESS_POP_10_15,CNTY10,% change
3,"Population, low access to store (%), 2010",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP10,CNTY10,Percent
4,"Population, low access to store (%), 2015",Access and Proximity to Grocery Store,ACCESS,Overall,PCT_LACCESS_POP15,CNTY10,Percent


In [162]:
len(df_food)

852810

In [165]:
df_food_mg = pd.merge(df_food, df_food_variables, on='Variable_Code',how='left')

In [180]:
df_food_mg.tail(50000)

,FIPS,State,County,Variable_Code,Value,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Geography,Units
802810,36019,NY,Clinton,PCT_NHASIAN10,1.083674,"% Asian, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Race & Age,CNTY10,Percent
802811,36019,NY,Clinton,PCT_NHNA10,0.323885,"% American Indian or Alaska Native, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Race & Age,CNTY10,Percent
802812,36019,NY,Clinton,PCT_NHPI10,0.018264,"% Hawaiian or Pacific Islander, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Race & Age,CNTY10,Percent
802813,36019,NY,Clinton,PCT_65OLDER10,13.335282,"% Population 65 years or older, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Race & Age,CNTY10,Percent
802814,36019,NY,Clinton,PCT_18YOUNGER10,19.347847,"% Population under age 18, 2010",Socioeconomic Characteristics,SOCIOECONOMIC,Race & Age,CNTY10,Percent
...,...,...,...,...,...,...,...,...,...,...,...
852805,56,WY,Total,State_Population_2014,582548.000000,NaN,NaN,NaN,NaN,NaN,NaN
852806,56,WY,Total,State_Population_2015,585668.000000,NaN,NaN,NaN,NaN,NaN,NaN
852807,56,WY,Total,State_Population_2016,584290.000000,NaN,NaN,NaN,NaN,NaN,NaN
852808,56,WY,Total,State_Population_2017,578934.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df_food_mg[df_food_mg.Variable_Code=='AGRITRSM_OPS07']

,FIPS,State,County,Variable_Code,Value,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Geography,Units
488276,1001,AL,Autauga,AGRITRSM_OPS07,7.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
488362,1003,AL,Baldwin,AGRITRSM_OPS07,18.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
488451,1005,AL,Barbour,AGRITRSM_OPS07,27.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
488528,1007,AL,Bibb,AGRITRSM_OPS07,5.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
488618,1009,AL,Blount,AGRITRSM_OPS07,10.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
...,...,...,...,...,...,...,...,...,...,...,...
741450,56037,WY,Sweetwater,AGRITRSM_OPS07,1.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
741522,56039,WY,Teton,AGRITRSM_OPS07,5.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
741594,56041,WY,Uinta,AGRITRSM_OPS07,5.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count
741658,56043,WY,Washakie,AGRITRSM_OPS07,8.0,"Agritourism operations, 2007",Local Foods,LOCAL,Other Local Food Outlets,CNTY10,Count


In [168]:
df_food_mg.Category_Name.unique()

array(['Access and Proximity to Grocery Store', nan, 'Store Availability',
       'Restaurant Availability and Expenditures', 'Food Assistance',
       'Food Insecurity', 'Food Taxes', 'Local Foods',
       'Health and Physical Activity', 'Socioeconomic Characteristics'],
      dtype=object)

In [169]:
df_food_mg.Category_Code.unique()

array(['ACCESS', nan, 'STORES', 'RESTAURANTS', 'ASSISTANCE', 'INSECURITY',
       'PRICES_TAXES', 'LOCAL', 'HEALTH', 'SOCIOECONOMIC'], dtype=object)

In [189]:
df_food_mg[df_food_mg.Variable_Code=='LACCESS_BLACK15']

,FIPS,State,County,Variable_Code,Value,Variable_Name,Category_Name,Category_Code,Subcategory_Name,Geography,Units
29,1001,AL,Autauga,LACCESS_BLACK15,4216.473194,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
70,1003,AL,Baldwin,LACCESS_BLACK15,3540.965826,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
111,1005,AL,Barbour,LACCESS_BLACK15,2713.124930,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
152,1007,AL,Bibb,LACCESS_BLACK15,419.099942,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
193,1009,AL,Blount,LACCESS_BLACK15,95.842687,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
...,...,...,...,...,...,...,...,...,...,...,...
128099,56037,WY,Sweetwater,LACCESS_BLACK15,186.278452,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
128140,56039,WY,Teton,LACCESS_BLACK15,9.089826,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
128181,56041,WY,Uinta,LACCESS_BLACK15,14.989817,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count
128222,56043,WY,Washakie,LACCESS_BLACK15,2.000000,"Black, low access to store, 2015",Access and Proximity to Grocery Store,ACCESS,Demographics,CNTY10,Count


In [181]:
# Pivot the table
df_pivot = df_food_mg.pivot_table(
    index=['FIPS', 'State', 'County'],  # Columns to keep and not unstack
    columns='Variable_Code',  # Column whose unique values will become new column names
    values='Value',  # Values to fill in the new columns
    aggfunc='first'  # Aggregate function in case of duplicate entries
)

# Reset the index to turn multi-index into columns
df_pivot.reset_index(inplace=True)

# Optionally, you can flatten the columns
#df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]


In [183]:
df_pivot.tail()

Variable_Code,FIPS,State,County,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,BERRY_ACRESPTH07,BERRY_ACRESPTH12,BERRY_FARMS07,BERRY_FARMS12,CACFP_PART_2012,CACFP_PART_2013,CACFP_PART_2014,CACFP_PART_2015,CACFP_PART_2016,CACFP_PART_2017,CACFP_PART_2018,CHILDPOVRATE15,CHIPSTAX_STORES14,CHIPSTAX_VENDM14,CH_FOODINSEC_14_17,CH_VLFOODSEC_14_17,CONVS11,CONVS16,CONVSPTH11,CONVSPTH16,CSA07,CSA12,DIRSALES07,DIRSALES12,DIRSALES_FARMS07,DIRSALES_FARMS12,FARM_TO_SCHOOL13,FARM_TO_SCHOOL15,FDPIR12,FDPIR15,FFR11,FFR16,FFRPTH11,FFRPTH16,FMRKT13,FMRKT18,FMRKTPTH13,FMRKTPTH18,FMRKT_ANMLPROD18,FMRKT_BAKED18,FMRKT_CREDIT18,FMRKT_FRVEG18,FMRKT_OTHERFOOD18,FMRKT_SFMNP18,FMRKT_SNAP18,FMRKT_WIC18,FMRKT_WICCASH18,FOODHUB18,FOODINSEC_12_14,FOODINSEC_15_17,FOOD_BANKS18,FOOD_TAX14,FRESHVEG_ACRES07,FRESHVEG_ACRES12,FRESHVEG_ACRESPTH07,FRESHVEG_ACRESPTH12,FRESHVEG_FARMS07,FRESHVEG_FARMS12,FSR11,FSR16,FSRPTH11,FSRPTH16,GHVEG_FARMS07,GHVEG_FARMS12,GHVEG_SQFT07,GHVEG_SQFT12,GHVEG_SQFTPTH07,GHVEG_SQFTPTH12,GROC11,GROC16,GROCPTH11,GROCPTH16,LACCESS_BLACK15,LACCESS_CHILD10,LACCESS_CHILD15,LACCESS_CHILD_10_15,LACCESS_HHNV10,LACCESS_HHNV15,LACCESS_HISP15,LACCESS_LOWI10,LACCESS_LOWI15,LACCESS_MULTIR15,LACCESS_NHASIAN15,LACCESS_NHNA15,LACCESS_NHPI15,LACCESS_POP10,LACCESS_POP15,LACCESS_SENIORS10,LACCESS_SENIORS15,LACCESS_SNAP15,LACCESS_WHITE15,MEDHHINC15,METRO13,NSLP_PART_2012,NSLP_PART_2013,NSLP_PART_2014,NSLP_PART_2015,NSLP_PART_2016,NSLP_PART_2017,NSLP_PART_2018,ORCHARD_ACRES07,ORCHARD_ACRES12,ORCHARD_ACRESPTH07,ORCHARD_ACRESPTH12,ORCHARD_FARMS07,ORCHARD_FARMS12,PCH_AGRITRSM_OPS_07_12,PCH_AGRITRSM_RCT_07_12,PCH_BERRY_ACRESPTH_07_12,PCH_BERRY_ACRES_07_12,PCH_BERRY_FARMS_07_12,PCH_CACFP_12_17,PCH_CONVSPTH_11_16,PCH_CONVS_11_16,PCH_CSA_07_12,PCH_DIRSALES_07_12,PCH_DIRSALES_FARMS_07_12,PCH_FDPIR_12_15,PCH_FFRPTH_11_16,PCH_FFR_11_16,PCH_FMRKTPTH_13_18,PCH_FMRKT_13_18,PCH_FRESHVEG_ACRESPTH_07_12,PCH_FRESHVEG_ACRES_07_12,PCH_FRESHVEG_FARMS_07_12,PCH_FSRPTH_11_16,PCH_FSR_11_16,PCH_GHVEG_FARMS_07_12,PCH_GHVEG_SQFTPTH_07_12,PCH_GHVEG_SQFT_07_12,PCH_GROCPTH_11_16,PCH_GROC_11_16,PCH_LACCESS_HHNV_10_15,PCH_LACCESS_LOWI_10_15,PCH_LACCESS_POP_10_15,PCH_LACCESS_SENIORS_10_15,PCH_NSLP_12_17,PCH_ORCHARD_ACRESPTH_07_12,PCH_ORCHARD_ACRES_07_12,PCH_ORCHARD_FARMS_07_12,PCH_PC_DIRSALES_07_12,PCH_PC_SNAPBEN_12_17,PCH_PC_WIC_REDEMP_11_16,PCH_RECFACPTH_11_16,PCH_RECFAC_11_16,PCH_REDEMP_SNAPS_12_17,PCH_REDEMP_WICS_11_16,PCH_SBP_12_17,PCH_SFSP_12_17,PCH_SLHOUSE_07_12,PCH_SNAPSPTH_12_17,PCH_SNAPS_12_17,PCH_SNAP_12_17,PCH_SPECSPTH_11_16,PCH_SPECS_11_16,PCH_SUPERCPTH_11_16,PCH_SUPERC_11_16,PCH_VEG_ACRESPTH_07_12,PCH_VEG_ACRES_07_12,PCH_VEG_FARMS_07_12,PCH_WICINFANTCHILD_14_16,PCH_WICSPTH_11_16,PCH_WICS_11_16,PCH_WICWOMEN_14_16,PCH_WIC_12_17,PCT_18YOUNGER10,PCT_65OLDER10,PCT_CACFP12,PCT_CACFP17,PCT_DIABETES_ADULTS08,PCT_DIABETES_ADULTS13,PCT_FMRKT_ANMLPROD18,PCT_FMRKT_BAKED18,PCT_FMRKT_CREDIT18,PCT_FMRKT_FRVEG18,PCT_FMRKT_OTHERFOOD18,PCT_FMRKT_SFMNP18,PCT_FMRKT_SNAP18,PCT_FMRKT_WIC18,PCT_FMRKT_WICCASH18,PCT_FREE_LUNCH10,PCT_FREE_LUNCH15,PCT_HISP10,PCT_HSPA17,PCT_LACCESS_BLACK15,PCT_LACCESS_CHILD10,PCT_LACCESS_CHILD15,PCT_LACCESS_HHNV10,PCT_LACCESS_HHNV15,PCT_LACCESS_HISP15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,PCT_LACCESS_MULTIR15,PCT_LACCESS_NHASIAN15,PCT_LACCESS_NHNA15,PCT_LACCESS_NHPI15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCT_LACCESS_SENIORS10,PCT_LACCESS_SENIORS15,PCT_LACCESS_SNAP15,PCT_LACCESS_WHITE15,PCT_LOCLFARM07,PCT_LOCLFARM12,PCT_LOCLSALE07,PCT_LOCLSALE12,PCT_NHASIAN10,PCT_NHBLACK10,PCT_NHNA10,PCT_NHPI10,PCT_NHWHITE10,PCT_NSLP12,PCT_NSLP17,PCT_OBESE_ADULTS12,PCT_OBESE_ADULTS17,PCT_REDUCED_LUNCH10,PCT_REDUCED_LUNCH15,PCT_SBP12,PCT_SBP17,PCT_SFSP12,PCT_SFSP17,PCT_SNAP12,PCT_SNAP17,PCT_WIC12,PCT_WIC17,PCT_WICINFANTCHILD14,PCT_WICINFANTCHILD16,PCT_WICWOMEN14,PCT_WICWOMEN16,PC_DIRSALES07,PC_DIRSALES12,PC_FFRSALES07,PC_FFRSALES12,PC_FSRSALES07,PC_FSRSALES12,PC_SNAPBEN12,PC_SNAPBEN17,PC_WIC_REDEMP11,PC_WIC_REDEMP16,PERCHLDPOV10,P

In [184]:
df_pivot[df_pivot.County=='Uinta']

Variable_Code,FIPS,State,County,2010_Census_Population,AGRITRSM_OPS07,AGRITRSM_OPS12,AGRITRSM_RCT07,AGRITRSM_RCT12,BERRY_ACRES07,BERRY_ACRES12,BERRY_ACRESPTH07,BERRY_ACRESPTH12,BERRY_FARMS07,BERRY_FARMS12,CACFP_PART_2012,CACFP_PART_2013,CACFP_PART_2014,CACFP_PART_2015,CACFP_PART_2016,CACFP_PART_2017,CACFP_PART_2018,CHILDPOVRATE15,CHIPSTAX_STORES14,CHIPSTAX_VENDM14,CH_FOODINSEC_14_17,CH_VLFOODSEC_14_17,CONVS11,CONVS16,CONVSPTH11,CONVSPTH16,CSA07,CSA12,DIRSALES07,DIRSALES12,DIRSALES_FARMS07,DIRSALES_FARMS12,FARM_TO_SCHOOL13,FARM_TO_SCHOOL15,FDPIR12,FDPIR15,FFR11,FFR16,FFRPTH11,FFRPTH16,FMRKT13,FMRKT18,FMRKTPTH13,FMRKTPTH18,FMRKT_ANMLPROD18,FMRKT_BAKED18,FMRKT_CREDIT18,FMRKT_FRVEG18,FMRKT_OTHERFOOD18,FMRKT_SFMNP18,FMRKT_SNAP18,FMRKT_WIC18,FMRKT_WICCASH18,FOODHUB18,FOODINSEC_12_14,FOODINSEC_15_17,FOOD_BANKS18,FOOD_TAX14,FRESHVEG_ACRES07,FRESHVEG_ACRES12,FRESHVEG_ACRESPTH07,FRESHVEG_ACRESPTH12,FRESHVEG_FARMS07,FRESHVEG_FARMS12,FSR11,FSR16,FSRPTH11,FSRPTH16,GHVEG_FARMS07,GHVEG_FARMS12,GHVEG_SQFT07,GHVEG_SQFT12,GHVEG_SQFTPTH07,GHVEG_SQFTPTH12,GROC11,GROC16,GROCPTH11,GROCPTH16,LACCESS_BLACK15,LACCESS_CHILD10,LACCESS_CHILD15,LACCESS_CHILD_10_15,LACCESS_HHNV10,LACCESS_HHNV15,LACCESS_HISP15,LACCESS_LOWI10,LACCESS_LOWI15,LACCESS_MULTIR15,LACCESS_NHASIAN15,LACCESS_NHNA15,LACCESS_NHPI15,LACCESS_POP10,LACCESS_POP15,LACCESS_SENIORS10,LACCESS_SENIORS15,LACCESS_SNAP15,LACCESS_WHITE15,MEDHHINC15,METRO13,NSLP_PART_2012,NSLP_PART_2013,NSLP_PART_2014,NSLP_PART_2015,NSLP_PART_2016,NSLP_PART_2017,NSLP_PART_2018,ORCHARD_ACRES07,ORCHARD_ACRES12,ORCHARD_ACRESPTH07,ORCHARD_ACRESPTH12,ORCHARD_FARMS07,ORCHARD_FARMS12,PCH_AGRITRSM_OPS_07_12,PCH_AGRITRSM_RCT_07_12,PCH_BERRY_ACRESPTH_07_12,PCH_BERRY_ACRES_07_12,PCH_BERRY_FARMS_07_12,PCH_CACFP_12_17,PCH_CONVSPTH_11_16,PCH_CONVS_11_16,PCH_CSA_07_12,PCH_DIRSALES_07_12,PCH_DIRSALES_FARMS_07_12,PCH_FDPIR_12_15,PCH_FFRPTH_11_16,PCH_FFR_11_16,PCH_FMRKTPTH_13_18,PCH_FMRKT_13_18,PCH_FRESHVEG_ACRESPTH_07_12,PCH_FRESHVEG_ACRES_07_12,PCH_FRESHVEG_FARMS_07_12,PCH_FSRPTH_11_16,PCH_FSR_11_16,PCH_GHVEG_FARMS_07_12,PCH_GHVEG_SQFTPTH_07_12,PCH_GHVEG_SQFT_07_12,PCH_GROCPTH_11_16,PCH_GROC_11_16,PCH_LACCESS_HHNV_10_15,PCH_LACCESS_LOWI_10_15,PCH_LACCESS_POP_10_15,PCH_LACCESS_SENIORS_10_15,PCH_NSLP_12_17,PCH_ORCHARD_ACRESPTH_07_12,PCH_ORCHARD_ACRES_07_12,PCH_ORCHARD_FARMS_07_12,PCH_PC_DIRSALES_07_12,PCH_PC_SNAPBEN_12_17,PCH_PC_WIC_REDEMP_11_16,PCH_RECFACPTH_11_16,PCH_RECFAC_11_16,PCH_REDEMP_SNAPS_12_17,PCH_REDEMP_WICS_11_16,PCH_SBP_12_17,PCH_SFSP_12_17,PCH_SLHOUSE_07_12,PCH_SNAPSPTH_12_17,PCH_SNAPS_12_17,PCH_SNAP_12_17,PCH_SPECSPTH_11_16,PCH_SPECS_11_16,PCH_SUPERCPTH_11_16,PCH_SUPERC_11_16,PCH_VEG_ACRESPTH_07_12,PCH_VEG_ACRES_07_12,PCH_VEG_FARMS_07_12,PCH_WICINFANTCHILD_14_16,PCH_WICSPTH_11_16,PCH_WICS_11_16,PCH_WICWOMEN_14_16,PCH_WIC_12_17,PCT_18YOUNGER10,PCT_65OLDER10,PCT_CACFP12,PCT_CACFP17,PCT_DIABETES_ADULTS08,PCT_DIABETES_ADULTS13,PCT_FMRKT_ANMLPROD18,PCT_FMRKT_BAKED18,PCT_FMRKT_CREDIT18,PCT_FMRKT_FRVEG18,PCT_FMRKT_OTHERFOOD18,PCT_FMRKT_SFMNP18,PCT_FMRKT_SNAP18,PCT_FMRKT_WIC18,PCT_FMRKT_WICCASH18,PCT_FREE_LUNCH10,PCT_FREE_LUNCH15,PCT_HISP10,PCT_HSPA17,PCT_LACCESS_BLACK15,PCT_LACCESS_CHILD10,PCT_LACCESS_CHILD15,PCT_LACCESS_HHNV10,PCT_LACCESS_HHNV15,PCT_LACCESS_HISP15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,PCT_LACCESS_MULTIR15,PCT_LACCESS_NHASIAN15,PCT_LACCESS_NHNA15,PCT_LACCESS_NHPI15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,PCT_LACCESS_SENIORS10,PCT_LACCESS_SENIORS15,PCT_LACCESS_SNAP15,PCT_LACCESS_WHITE15,PCT_LOCLFARM07,PCT_LOCLFARM12,PCT_LOCLSALE07,PCT_LOCLSALE12,PCT_NHASIAN10,PCT_NHBLACK10,PCT_NHNA10,PCT_NHPI10,PCT_NHWHITE10,PCT_NSLP12,PCT_NSLP17,PCT_OBESE_ADULTS12,PCT_OBESE_ADULTS17,PCT_REDUCED_LUNCH10,PCT_REDUCED_LUNCH15,PCT_SBP12,PCT_SBP17,PCT_SFSP12,PCT_SFSP17,PCT_SNAP12,PCT_SNAP17,PCT_WIC12,PCT_WIC17,PCT_WICINFANTCHILD14,PCT_WICINFANTCHILD16,PCT_WICWOMEN14,PCT_WICWOMEN16,PC_DIRSALES07,PC_DIRSALES12,PC_FFRSALES07,PC_FFRSALES12,PC_FSRSALES07,PC_FSRSALES12,PC_SNAPBEN12,PC_SNAPBEN17,PC_WIC_REDEMP11,PC_WIC_REDEMP16,PERCHLDPOV10,P

In [186]:
len(df_pivot.columns)

335

In [ ]:
# I don't think that's what I want. I think that's too much info. What I think we should do is pull out a few interesting variables and add them to the one row per State+County df.
# other info could be available on drill down? like I don't want to provide a drop down menu of 330 variables to color code the map by.
# 

In [191]:
df_spend = pd.read_csv("../data/raw/FY2017_All_Assistance_Full_20231220/FY2017_All_Assistance_Full_20231220_1.csv")

/tmp/ipykernel_436/2503402599.py:1: DtypeWarning: Columns (34,54,66,67,80,83,84,99,101,103,105,107) have mixed types. Specify dtype option on import or set low_memory=False.
  df_spend = pd.read_csv("../data/raw/FY2017_All_Assistance_Full_20231220/FY2017_All_Assistance_Full_20231220_1.csv")


In [192]:
df_spend.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,non_federal_funding_amount,total_non_federal_funding_amount,face_value_of_loan,original_loan_subsidy_cost,total_face_value_of_loan,total_loan_subsidy_cost,generated_pragmatic_obligations,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_from_COVID-19_supplementals_for_overall_award,obligated_amount_from_COVID-19_supplementals_for_overall_award,outlayed_amount_from_IIJA_supplemental_for_overall_award,obligated_amount_from_IIJA_supplemental_for_overall_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,recipient_uei,recipient_duns,recipient_name,recipient_name_raw,recipient_parent_uei,recipient_parent_duns,recipient_parent_name,recipient_parent_name_raw,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_code,recipient_city_name,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_recipient_state_fips_code,recipient_state_code,recipient_state_name,recipient_zip_code,recipient_zip_last_4_code,prime_award_transaction_recipient_cd_original,prime_award_transaction_recipient_cd_current,recipient_foreign_city_name,recipient_foreign_province_name,recipient_foreign_postal_code,primary_place_of_performance_scope,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_code,primary_place_of_performance_city_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name,prime_award_transaction_place_of_performance_state_fips_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,prime_award_transaction_place_of_performance_cd_original,prime_award_transaction_place_of_performance_cd_current,primary_place_of_performance_foreign_location,cfda_number,cfda_title,funding_opportunity_number,funding_opportunity_goals_text,assistance_type_code,assistance_type_description,transaction_description,prime_award_base_transaction_description,business_funds_indicator_code,business_funds_indicator_description,business_types_code,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,3620_-NONE-_18120424.0123118_64.012_-NONE-,ASST_AGG_18120424.0123118_3620,NaN,NaN,18120424.0123118,NaN,20932.0,20932.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,20932.0,NaN,NaN,NaN,NaN,NaN,2017-09-30,2017,NaN,NaN,36,Department of Veterans Affairs,3620,Under Secretary for Health/Veterans Health Adm...,NaN,NaN,36.0,Department of Veterans Affairs,3600,Department of Veterans Affairs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTIPLE RECIPIENTS,MULTIPLE RECIPIENTS,NaN,NaN,NaN,NaN,USA,UNITED STATES,NaN,NaN,NaN,NaN,72087.0,LOÍZA,72.0,PR,PUERTO RICO,NaN,NaN,PR-98,PR-98,NaN,NaN,NaN,County-wide,USA,UNITED STATES,PR**087,NaN,72087.0,LOÍZA,72.0,PUERTO RICO,NaN,

In [193]:
len(df_spend)

1000000

In [194]:
len(df_spend.columns)

111

In [197]:
for col in df_spend.columns:
    print(col)

assistance_transaction_unique_key
assistance_award_unique_key
award_id_fain
modification_number
award_id_uri
sai_number
federal_action_obligation
total_obligated_amount
total_outlayed_amount_for_overall_award
indirect_cost_federal_share_amount
non_federal_funding_amount
total_non_federal_funding_amount
face_value_of_loan
original_loan_subsidy_cost
total_face_value_of_loan
total_loan_subsidy_cost
generated_pragmatic_obligations
disaster_emergency_fund_codes_for_overall_award
outlayed_amount_from_COVID-19_supplementals_for_overall_award
obligated_amount_from_COVID-19_supplementals_for_overall_award
outlayed_amount_from_IIJA_supplemental_for_overall_award
obligated_amount_from_IIJA_supplemental_for_overall_award
action_date
action_date_fiscal_year
period_of_performance_start_date
period_of_performance_current_end_date
awarding_agency_code
awarding_agency_name
awarding_sub_agency_code
awarding_sub_agency_name
awarding_office_code
awarding_office_name
funding_agency_code
funding_agency_name

In [198]:
df_contracts = pd.read_csv("../data/raw/FY2017_All_Contracts_Full_20231220/FY2017_All_Contracts_Full_20231220_5.csv")

/tmp/ipykernel_436/2993895985.py:1: DtypeWarning: Columns (5,27,47,71,72,81,115,135,136,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df_contracts = pd.read_csv("../data/raw/FY2017_All_Contracts_Full_20231220/FY2017_All_Contracts_Full_20231220_5.csv")


In [199]:
df_contracts.head()

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,total_outlayed_amount_for_overall_award,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_from_COVID-19_supplementals_for_overall_award,obligated_amount_from_COVID-19_supplementals_for_overall_award,outlayed_amount_from_IIJA_supplemental_for_overall_award,obligated_amount_from_IIJA_supplemental_for_overall_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_uei,recipient_duns,recipient_name,recipient_name_raw,recipient_doing_business_as_name,cage_code,recipient_parent_uei,recipient_parent_duns,recipient_parent_name,recipient_parent_name_raw,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_recipient_state_fips_code,recipient_state_code,recipient_state_name,recipient_zip_4_code,prime_award_transaction_recipient_cd_original,prime_award_transaction_recipient_cd_current,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name,prime_award_transaction_place_of_performance_state_fips_code,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,prime_award_transaction_place_of_performance_cd_original,prime_award_transaction_place_of_performance_cd_current,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,transaction_description,prime_award_base_transaction_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,

In [200]:
len(df_contracts)

912914

In [201]:
len(df_contracts.columns)

296

In [203]:
for col in df_contracts.columns:
    print(col)

contract_transaction_unique_key
contract_award_unique_key
award_id_piid
modification_number
transaction_number
parent_award_agency_id
parent_award_agency_name
parent_award_id_piid
parent_award_modification_number
federal_action_obligation
total_dollars_obligated
total_outlayed_amount_for_overall_award
base_and_exercised_options_value
current_total_value_of_award
base_and_all_options_value
potential_total_value_of_award
disaster_emergency_fund_codes_for_overall_award
outlayed_amount_from_COVID-19_supplementals_for_overall_award
obligated_amount_from_COVID-19_supplementals_for_overall_award
outlayed_amount_from_IIJA_supplemental_for_overall_award
obligated_amount_from_IIJA_supplemental_for_overall_award
action_date
action_date_fiscal_year
period_of_performance_start_date
period_of_performance_current_end_date
period_of_performance_potential_end_date
ordering_period_end_date
solicitation_date
awarding_agency_code
awarding_agency_name
awarding_sub_agency_code
awarding_sub_agency_name
award

In [206]:
[x for x in df_contracts.columns if 'county' in x.lower()]

['prime_award_transaction_recipient_county_fips_code',
 'recipient_county_name',
 'prime_award_transaction_place_of_performance_county_fips_code',
 'primary_place_of_performance_county_name',
 'county_local_government']

In [210]:
df_contracts[['prime_award_transaction_recipient_county_fips_code',
 'recipient_county_name',
 'prime_award_transaction_place_of_performance_county_fips_code',
 'primary_place_of_performance_county_name']].head()

,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name
0,47157.0,SHELBY,47157.0,SHELBY
1,29189.0,ST. LOUIS,29189.0,SAINT LOUIS
2,34005.0,BURLINGTON,34005.0,BURLINGTON
3,12011.0,BROWARD,12011.0,BROWARD
4,12091.0,OKALOOSA,12091.0,OKALOOSA


In [207]:
[x for x in df_spend.columns if 'county' in x.lower()]

['prime_award_transaction_recipient_county_fips_code',
 'recipient_county_name',
 'prime_award_transaction_place_of_performance_county_fips_code',
 'primary_place_of_performance_county_name']

In [209]:
df_spend[['prime_award_transaction_recipient_county_fips_code',
 'recipient_county_name',
 'prime_award_transaction_place_of_performance_county_fips_code',
 'primary_place_of_performance_county_name']].head()

,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name
0,72087.0,LOÍZA,72087.0,LOÍZA
1,26055.0,GRAND TRAVERSE,26055.0,GRAND TRAVERSE
2,15007.0,KAUAI,15007.0,KAUAI
3,37071.0,GASTON,37071.0,GASTON
4,1045.0,DALE,1045.0,DALE


## USA Spending API stuff

In [211]:
import requests
import json

# API endpoint
url = 'https://api.usaspending.gov/api/v2/search/spending_by_geography/'

# Query payload
payload = {
  "filters": {
    "time_period": [
      {
        "start_date": "2017-10-01",
        "end_date": "2018-09-30"
      }
    ],
    "geo_layer": "county",
    "geo_layer_filters": [
      {
        "scope": "state_code",
        "layer": "state",
        "name": "TN"
      }
    ]
  },
  "fields": ["County Name", "Total Obligations"],
  "sort": "Total Obligations",
  "order": "desc"
}




In [212]:
# Send the POST request
response = requests.post(url, json=payload)

# Parse the response
data = response.json()

# Show a snippet of the response
data_snippet = data['results'][:5]  # Show first 5 entries as a snippet
data_snippet


KeyError: 'results'

In [215]:
import requests

url = "https://api.usaspending.gov/api/v2/search/spending_by_geography/"
headers = {
    'Content-Type': 'application/json'
}
data = {
    "scope": "place_of_performance",
    "geo_layer": "county",
    "geo_layer_filter": ["TN"],
    "filters": {
        "time_period": [{"start_date": "2017-01-01", "end_date": "2017-12-31"}]
    }
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    print("Request successful")
    print(response.json())  # This will print the JSON response
else:
    print("Request failed")
    print(response.status_code)
    print(response.text)

# Assuming 'response_json' is the JSON response you received
response_json = response.json()

# Extract the 'results' data
data = response_json['results']

# Convert to DataFrame
df = pd.DataFrame(data)

Request successful
{'scope': 'place_of_performance', 'geo_layer': 'county', 'results': [{'shape_code': '42029', 'display_name': 'Chester County', 'aggregated_amount': 4496307531.41, 'population': 534413, 'per_capita': 8413.54}, {'shape_code': '17097', 'display_name': 'Lake County', 'aggregated_amount': 3923439852.34, 'population': 714342, 'per_capita': 5492.38}, {'shape_code': '34013', 'display_name': 'Essex County', 'aggregated_amount': 4667456485.88, 'population': 863728, 'per_capita': 5403.85}, {'shape_code': '51085', 'display_name': 'Hanover County', 'aggregated_amount': 661840817.3, 'population': 109979, 'per_capita': 6017.88}, {'shape_code': '34005', 'display_name': 'Burlington County', 'aggregated_amount': 4074993515.08, 'population': 461860, 'per_capita': 8823.01}, {'shape_code': '06037', 'display_name': 'Los Angeles County', 'aggregated_amount': 60350542300.97, 'population': 10014009, 'per_capita': 6026.61}, {'shape_code': '11001', 'display_name': 'District Of Columbia', 'aggr

In [216]:
# Assuming 'response_json' is the JSON response you received
response_json = response.json()

# Extract the 'results' data
data = response_json['results']

# Convert to DataFrame
df = pd.DataFrame(data)

In [217]:
df.head()

,shape_code,display_name,aggregated_amount,population,per_capita
0,42029,Chester County,4.496308e+09,534413.0,8413.54
1,17097,Lake County,3.923440e+09,714342.0,5492.38
2,34013,Essex County,4.667456e+09,863728.0,5403.85
3,51085,Hanover County,6.618408e+08,109979.0,6017.88
4,34005,Burlington County,4.074994e+09,461860.0,8823.01


In [229]:
import requests
import pandas as pd

def fetch_spending_data_by_county(fiscal_year, state_code):
    url = "https://api.usaspending.gov/api/v2/search/spending_by_geography/"
    headers = {
        'Content-Type': 'application/json'
    }
    data = {
        "scope": "place_of_performance",
        "geo_layer": "county",
        "geo_layer_filter": [state_code],
        "filters": {
            "time_period": [{"start_date": f"{fiscal_year}-01-01", "end_date": f"{fiscal_year}-12-31"}]
        }
    }

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        print("Request successful")
        response_json = response.json()

        # Extract the 'results' data
        data = response_json['results']

        # Convert to DataFrame
        df = pd.DataFrame(data)

        # Add fiscal year and state code to the DataFrame
        df['fiscal_year'] = fiscal_year
        df['state_code'] = state_code

        return df
    else:
        print("Request failed")
        print(response.status_code)
        print(response.text)
        return pd.DataFrame()




In [230]:
# Example usage
df_county_spending = fetch_spending_data_by_county(2017, "TN")
df_county_spending.head()

Request successful


,shape_code,display_name,aggregated_amount,population,per_capita,fiscal_year,state_code
0,42029,Chester County,4.496308e+09,534413.0,8413.54,2017,TN
1,17097,Lake County,3.923440e+09,714342.0,5492.38,2017,TN
2,34013,Essex County,4.667456e+09,863728.0,5403.85,2017,TN
3,51085,Hanover County,6.618408e+08,109979.0,6017.88,2017,TN
4,34005,Burlington County,4.074994e+09,461860.0,8823.01,2017,TN


In [233]:
import requests
import pandas as pd

def fetch_spending_data_by_county(fiscal_year, state_codes):
    url = "https://api.usaspending.gov/api/v2/search/spending_by_geography/"
    headers = {'Content-Type': 'application/json'}
    all_data = []

    for state_code in state_codes:
        data = {
            "scope": "place_of_performance",
            "geo_layer": "county",
            "geo_layer_filter": [state_code],
            "filters": {
                "time_period": [{"start_date": f"{fiscal_year}-01-01", "end_date": f"{fiscal_year}-12-31"}]
            }
        }
        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            print(f"Request successful for {state_code}")
            response_json = response.json()
            data = response_json['results']
            for d in data:
                d['fiscal_year'] = fiscal_year
                d['state_code'] = state_code
            all_data.extend(data)
        else:
            print(f"Request failed for {state_code}")
            print(response.status_code)
            print(response.text)

    return pd.DataFrame(all_data)

# Example usage
df = fetch_spending_data_by_county(2017, ["TN", "VA"])
df.head()

Request successful for TN
Request successful for VA


,shape_code,display_name,aggregated_amount,population,per_capita,fiscal_year,state_code
0,42029,Chester County,4.496308e+09,534413.0,8413.54,2017,TN
1,17097,Lake County,3.923440e+09,714342.0,5492.38,2017,TN
2,34013,Essex County,4.667456e+09,863728.0,5403.85,2017,TN
3,51085,Hanover County,6.618408e+08,109979.0,6017.88,2017,TN
4,34005,Burlington County,4.074994e+09,461860.0,8823.01,2017,TN


In [238]:
df.tail(20)

,shape_code,display_name,aggregated_amount,population,per_capita,fiscal_year,state_code
6446,31075,Grant County,3301910.28,611.0,5404.11,2017,VA
6447,30103,Treasure County,8891532.34,762.0,11668.68,2017,VA
6448,31117,Mcpherson County,1592524.32,399.0,3991.29,2017,VA
6449,02013,Aleutians East Borough,16224552.85,3420.0,4744.02,2017,VA
6450,30069,Petroleum County,1998650.24,496.0,4029.54,2017,VA
6451,02282,Yakutat City And Borough,6513448.91,662.0,9839.05,2017,VA
6452,31183,Wheeler County,3943832.77,774.0,5095.39,2017,VA
6453,48393,Roberts County,2587591.83,827.0,3128.89,2017,VA
6454,02158,Kusilvak Census Area,8485554.68,8368.0,1014.05,2017,VA
6455,31005,Arthur County,1439539.72,434.0,3316.91,2017,VA


In [239]:
df[df.state_code=='TN'].tail(20)

,shape_code,display_name,aggregated_amount,population,per_capita,fiscal_year,state_code
3213,31075,Grant County,3301910.28,611.0,5404.11,2017,TN
3214,30103,Treasure County,8891532.34,762.0,11668.68,2017,TN
3215,31117,Mcpherson County,1592524.32,399.0,3991.29,2017,TN
3216,02013,Aleutians East Borough,16224552.85,3420.0,4744.02,2017,TN
3217,30069,Petroleum County,1998650.24,496.0,4029.54,2017,TN
3218,02282,Yakutat City And Borough,6513448.91,662.0,9839.05,2017,TN
3219,31183,Wheeler County,3943832.77,774.0,5095.39,2017,TN
3220,48393,Roberts County,2587591.83,827.0,3128.89,2017,TN
3221,02158,Kusilvak Census Area,8485554.68,8368.0,1014.05,2017,TN
3222,31005,Arthur County,1439539.72,434.0,3316.91,2017,TN


In [240]:
len(df)

6466

In [241]:
df

,shape_code,display_name,aggregated_amount,population,per_capita,fiscal_year,state_code
0,42029,Chester County,4.496308e+09,534413.0,8413.54,2017,TN
1,17097,Lake County,3.923440e+09,714342.0,5492.38,2017,TN
2,34013,Essex County,4.667456e+09,863728.0,5403.85,2017,TN
3,51085,Hanover County,6.618408e+08,109979.0,6017.88,2017,TN
4,34005,Burlington County,4.074994e+09,461860.0,8823.01,2017,TN
...,...,...,...,...,...,...,...
6461,69085,Northern Islands Municipality,1.919505e+07,7.0,2742150.57,2017,VA
6462,69120,Tinian Municipality,1.370485e+07,2044.0,6704.92,2017,VA
6463,69100,Rota Municipality,4.595719e+06,1893.0,2427.74,2017,VA
6464,60020,Manu'A District,1.218614e+07,832.0,14646.80,2017,VA


In [243]:
import requests
import pandas as pd
import us

def fetch_spending_data_by_county(fiscal_year):
    url = "https://api.usaspending.gov/api/v2/search/spending_by_geography/"
    headers = {
        'Content-Type': 'application/json'
    }
    data = {
        "scope": "place_of_performance",
        "geo_layer": "county",
        "filters": {
            "time_period": [{"start_date": f"{fiscal_year}-01-01", "end_date": f"{fiscal_year}-12-31"}]
        }
    }

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        print("Request successful")
        response_json = response.json()
        results = response_json['results']

        for result in results:
            fips_code = result['shape_code']
            state = us.states.lookup(fips_code[:2])
            result['state_code'] = state.abbr if state else None

        df = pd.DataFrame(results)
        df['fiscal_year'] = fiscal_year

        return df
    else:
        print("Request failed")
        print(response.status_code)
        print(response.text)
        return pd.DataFrame()

# Example usage
df = fetch_spending_data_by_county(2017)


Request successful


In [244]:
df

,shape_code,display_name,aggregated_amount,population,per_capita,state_code,fiscal_year
0,42029,Chester County,4.496308e+09,534413.0,8413.54,PA,2017
1,17097,Lake County,3.923440e+09,714342.0,5492.38,IL,2017
2,34013,Essex County,4.667456e+09,863728.0,5403.85,NJ,2017
3,51085,Hanover County,6.618408e+08,109979.0,6017.88,VA,2017
4,34005,Burlington County,4.074994e+09,461860.0,8823.01,NJ,2017
...,...,...,...,...,...,...,...
3228,69085,Northern Islands Municipality,1.919505e+07,7.0,2742150.57,MP,2017
3229,69120,Tinian Municipality,1.370485e+07,2044.0,6704.92,MP,2017
3230,69100,Rota Municipality,4.595719e+06,1893.0,2427.74,MP,2017
3231,60020,Manu'A District,1.218614e+07,832.0,14646.80,AS,2017


In [247]:
df[df['state_code'].isna()]

,shape_code,display_name,aggregated_amount,population,per_capita,state_code,fiscal_year
6,11001,District Of Columbia,3.008526e+10,689545.0,43630.6,None,2017
2673,,None,0.000000e+00,NaN,NaN,None,2017
